In [1]:
import pandas as pd
import cv2
import subprocess
from functools import partial
from collections import defaultdict
from sklearn.model_selection import train_test_split
import numpy as np
from tqdm import tqdm
import os
import sys

In [1]:
from tensorflow.python.client import device_lib
def get_available_devices():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos]
print(get_available_devices()) 

['/device:CPU:0', '/device:GPU:0']


In [2]:
data = pd.read_excel('../Downloads/Telegram Desktop/разметка.xlsx')
data.head()

,File,Label,Start_,End_,Special_transport_
0,1,1,00:00:06,00:00:14,1
1,2,1,00:00:00,00:00:58,1
2,3,0,00:00:00,00:00:00,0
3,4,1,00:00:03,00:00:13,0
4,5,1,00:00:02,00:00:12,1


In [3]:
save_path = '../Downloads/dtp_images'
subprocess.Popen(f"mkdir {save_path}").wait()

1

In [4]:
rs = 1

In [5]:
train_data, val_test_data = train_test_split(data, test_size = 10, stratify = data.Label, random_state = rs)
val_data, test_data = train_test_split(val_test_data, test_size = 5, stratify = val_test_data.Label, random_state = rs)

In [6]:
X_train, y_train = [], []
X_val, y_val = [], []

In [30]:
def preprocess_videos(resize_shape = (224, 224), data_dir = '../Downloads/Онлайн этап/'):
    for filename in list(os.walk(data_dir))[0][2]:
        vidcap = cv2.VideoCapture(data_dir+filename)
        int_filename = int(filename.split('.')[0])
        if int_filename in train_data.File.values:
            target_data = X_train
            y_train.append(train_data[train_data.File==int_filename].Label)
        elif int_filename in val_data.File.values:
            target_data = X_val
            y_val.append(val_data[val_data.File==int_filename].Label)
        else:
            continue
        target_data.append([])
        count = 0
        while True:
            ret, frame = vidcap.read()
            if not ret:
                break
            if not count % 10:
                resized_frame = cv2.resize(frame, resize_shape, interpolation = cv2.INTER_AREA)
                #cv2.imwrite(f"{save_path}/{filename.split('.')[0]}_{count}.png", resized_frame)
                target_data[-1].append(resized_frame)
            count += 1
        vidcap.release()
        target_data[-1] = np.array(target_data[-1])
        
        print(f"{filename} has been processed")

In [8]:
preprocess_videos()

10.mp4 has been processed
11.mp4 has been processed
12.mp4 has been processed
13.mp4 has been processed
14.mp4 has been processed
15.mp4 has been processed
16.mp4 has been processed
17.mp4 has been processed
18.mp4 has been processed
19.mp4 has been processed
2.mp4 has been processed
20.mp4 has been processed
21.mp4 has been processed
22.mp4 has been processed
23.mp4 has been processed
24.mp4 has been processed
26.mp4 has been processed
27.mp4 has been processed
28.mp4 has been processed
3.mp4 has been processed
31.mp4 has been processed
32.mp4 has been processed
33.mp4 has been processed
34.mp4 has been processed
35.mp4 has been processed
36.mp4 has been processed
37.mp4 has been processed
38.mp4 has been processed
39.mp4 has been processed
4.mp4 has been processed
40.mp4 has been processed
41.mp4 has been processed
42.mp4 has been processed
43.mp4 has been processed
44.mp4 has been processed
45.mp4 has been processed
46.mp4 has been processed
47.mp4 has been processed
48.mp4 has been

In [9]:
max_len = max(map(len, X_train))
max_len

389

In [16]:
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.applications.resnet_v2 import preprocess_input, ResNet152V2 
from tensorflow.keras.layers import Bidirectional, TimeDistributed, GRU, Dense, Input
from tensorflow.keras.models import Model

In [17]:
X_train = tf.keras.preprocessing.sequence.pad_sequences(
        X_train, maxlen=max_len, dtype='float16', padding='pre',
        truncating='pre', value=0.0
    )
X_val = tf.keras.preprocessing.sequence.pad_sequences(
        X_val, maxlen=max_len, dtype='float16', padding='pre',
        truncating='pre', value=0.0
    )

In [18]:
fps = 10

In [19]:
base_model = ResNet152V2(weights='imagenet')

In [20]:
for layer in base_model.layers:
    layer.trainable = False

In [21]:
inp = Input(shape = (max_len, 224, 224, 3), batch_size = 4)
resnet = TimeDistributed(base_model)(inp)
rnn = Bidirectional(GRU(64, return_sequences = True))(resnet)
rnn2 = Bidirectional(GRU(32))(rnn)
x = Dense(100, activation = 'relu')(rnn2)
otput = Dense(1)(x)

In [22]:
model = Model(inputs=inp, outputs=otput)

In [23]:
model.compile(optimizer='adam', loss=tf.keras.losses.BinaryCrossentropy(from_logits=True), metrics=['accuracy'])

In [24]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(4, 389, 224, 224, 3)]   0         
                                                                 
 time_distributed_1 (TimeDis  (4, 389, 1000)           60380648  
 tributed)                                                       
                                                                 
 bidirectional_2 (Bidirectio  (4, 389, 128)            409344    
 nal)                                                            
                                                                 
 bidirectional_3 (Bidirectio  (4, 64)                  31104     
 nal)                                                            
                                                                 
 dense_2 (Dense)             (4, 100)                  6500      
                                                             

In [25]:
y_train = np.array([i.values for i in y_train]).ravel()
y_val = np.array([i.values for i in y_val]).ravel()

In [29]:
y_val

array([], dtype=int64)

In [26]:
class BatchGenerator(object):
    def __init__(self, X: np.array, y: np.array, batch_size: int = 4):
        self.X = X
        self.y = y
        self.batch_size = batch_size
        
    def get_next_batch(self) -> tuple:
        indices = np.arange(len(self.X))
        np.random.shuffle(indices)
        batch_begin = 0
        while batch_begin < len(self.X):
            batch_indices = indices[batch_begin : batch_begin+self.batch_size]
            batch = tf.constant(self.X[(batch_indices)])
            labels = tf.constant(self.y[(batch_indices)])
            batch_begin += self.batch_size
            yield batch, labels

In [27]:
train_generator = BatchGenerator(X_train, y_train)
val_generator = BatchGenerator(X_val, y_val)

In [28]:
model.fit(x=train_generator.get_next_batch(), epochs=3, validation_data=val_generator.get_next_batch())

Epoch 1/3
     10/Unknown - 1095s 109s/step - loss: 0.6785 - accuracy: 0.6000

IndexError: index 0 is out of bounds for axis 0 with size 0

In [ ]:
base_model = TimeDistributed(ResNet152V2(weights='imagenet'))

# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(1024, activation='relu')(x)
# and a logistic layer -- let's say we have 200 classes
predictions = Dense(200, activation='softmax')(x)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False

# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')


In [17]:
x = tf.keras.applications.resnet_v2.preprocess_input(X_train)

In [18]:
x_val = tf.keras.applications.resnet_v2.preprocess_input(X_val)

In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
import cv2
import numpy as np
from keras import layers
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D
from keras.models import Model, load_model
from keras.initializers import glorot_uniform
# from keras.utils import plot_model
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
import keras.backend as K
import tensorflow as tf

In [37]:
model = tf.keras.applications.resnet_v2.ResNet152V2(
    include_top=True, weights=None, input_tensor=None,
    input_shape=None, pooling=None, classes=2,
    classifier_activation='softmax'
)

In [38]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [40]:
model.fit(x, y_train, epochs = 5, batch_size = 16)

InternalError: Failed copying input tensor from /job:localhost/replica:0/task:0/device:CPU:0 to /job:localhost/replica:0/task:0/device:GPU:0 in order to run _EagerConst: Dst tensor is not initialized.

In [5]:
FPS = 30

In [6]:
def get_frame_from_time(x):
    return x.second*FPS

In [67]:
data['Start_frame'] = data['Start_'].apply(get_frame_from_time)
data['End_frame'] = data['End_'].apply(get_frame_from_time)

In [68]:
data.head()

,File,Label,Start_,End_,Special_transport_,Start_frame,End_frame
0,1,1,00:00:06,00:00:14,1,180,420
1,2,1,00:00:00,00:00:58,1,0,1740
2,3,0,00:00:00,00:00:00,0,0,0
3,4,1,00:00:03,00:00:13,0,90,390
4,5,1,00:00:02,00:00:12,1,60,360


In [71]:
lengths = []

In [74]:
for i in range(1, 51):
    frames_count = 0
    for filename in list(os.walk(save_path))[0][2]:
        if filename.split('_')[0] == str(i):
            frames_count += 1
    lengths.append(frames_count)

In [76]:
max(lengths)

3884

In [81]:
data = data.set_index('File')

In [14]:
train_data, val_test_data = train_test_split(data, test_size = 10, stratify = data.Label)
val_data, test_data = train_test_split(val_test_data, test_size = 5, stratify = val_test_data.Label)

In [16]:
train_dict = {}
val_dict = {}
test_dict = {}

In [ ]:
for filename in list(os.walk('../Downloads/dtp_images'))[0][2]:
    video_name = filename.split('_')[0]
    if video_name in train_data.File.values:
        

In [ ]:
for i in range(1, 51):
    

In [11]:
ex = cv2.imread('../Downloads/dtp_images/1_0.png')

In [12]:
ex

array([[[ 62,  62,  62],
        [ 44,  44,  44],
        [ 38,  38,  38],
        ...,
        [133, 133, 133],
        [134, 134, 134],
        [132, 132, 132]],

       [[ 50,  50,  50],
        [ 44,  44,  44],
        [ 38,  38,  38],
        ...,
        [136, 136, 136],
        [137, 137, 137],
        [133, 133, 133]],

       [[ 48,  48,  48],
        [ 43,  43,  43],
        [ 38,  38,  38],
        ...,
        [136, 136, 136],
        [138, 138, 138],
        [135, 135, 135]],

       ...,

       [[ 18,  18,  18],
        [ 19,  19,  19],
        [ 24,  24,  24],
        ...,
        [248, 248, 248],
        [248, 248, 248],
        [248, 248, 248]],

       [[ 19,  19,  19],
        [ 19,  19,  19],
        [ 22,  22,  22],
        ...,
        [248, 248, 248],
        [248, 248, 248],
        [248, 248, 248]],

       [[ 20,  20,  20],
        [ 19,  19,  19],
        [ 22,  22,  22],
        ...,
        [248, 248, 248],
        [248, 248, 248],
        [248, 248, 248]]

In [93]:
data_labels = {0:[], 1:[]}
for filename in list(os.walk(save_path))[0][2]:
    image_name = filename.split('_')
    video_name = image_name[0]
    frame_number = image_name[1].split('.')[0]
    if data.loc[int(video_name), 'Label']:
        label = 1 if data.loc[int(video_name), 'Start_frame'] < int(frame_number) < data.loc[int(video_name), 'End_frame'] else 0
    else:
        label = 0
    data_labels[label].append(filename)

In [99]:
[len(data_labels[key]) for key in data_labels.keys()]

[41728, 6310]

In [91]:
21697+7068+12088+2403

43256

In [9]:
vidcap = cv2.VideoCapture('../Downloads/Онлайн этап/1.mp4')

In [15]:
success, image = vidcap.read()

In [16]:
image

array([[[ 67,  67,  67],
        [ 66,  66,  66],
        [ 60,  60,  60],
        ...,
        [133, 133, 133],
        [130, 130, 130],
        [132, 132, 132]],

       [[ 71,  71,  71],
        [ 69,  69,  69],
        [ 55,  55,  55],
        ...,
        [135, 135, 135],
        [132, 132, 132],
        [133, 133, 133]],

       [[ 59,  59,  59],
        [ 58,  58,  58],
        [ 48,  48,  48],
        ...,
        [137, 137, 137],
        [135, 135, 135],
        [133, 133, 133]],

       ...,

       [[ 19,  19,  19],
        [ 19,  19,  19],
        [ 19,  19,  19],
        ...,
        [248, 248, 248],
        [248, 248, 248],
        [248, 248, 248]],

       [[ 20,  20,  20],
        [ 20,  20,  20],
        [ 20,  20,  20],
        ...,
        [248, 248, 248],
        [248, 248, 248],
        [248, 248, 248]],

       [[ 20,  20,  20],
        [ 20,  20,  20],
        [ 20,  20,  20],
        ...,
        [248, 248, 248],
        [248, 248, 248],
        [248, 248, 248]]

In [ ]:
image = cv2.resize(image, size, cv.INTER_AREA)

In [13]:
image.shape

(1080, 1920, 3)